In [ ]:
from io import BytesIO
import hashlib
import random

import util
from test_framework.key import generate_schnorr_nonce, ECKey, ECPubKey, SECP256K1_FIELD_SIZE
from test_framework.musig import aggregate_musig_signatures, aggregate_schnorr_nonces, generate_musig_key, sign_musig
from test_framework.script import TapLeaf, TapTree, TaprootSignatureHash
from test_framework.address import program_to_witness
from test_framework.messages import CTransaction, COutPoint, CTxIn, CTxOut, CScriptWitness, CTxInWitness
from test_framework.util import assert_equal


# 3.1 Degrading Multisig Output

In this case-study, we consider a degrading multisig output, which provides recovery spending paths if the main wallet keys are lost or cannot sign. This output is expected to spent soon after being created. 
The recovery spending paths include delays in case the back-up keys are compromised.

***Locking conditions***

* `3/3 multisig`
    * Requires 3 main wallet keys
* Or `3/5 multsig` spendable after 3 days
    * Requires 1 backup key
* Or  `3/5 multisig` spendable after 10 days
    * Requires 2 backup keys

***Signers***

* Keys A, B, C - _main wallet keys_
* Keys D, E - _backup keys_

***Privacy Requirements***

* No unused public keys should be revealed during spending.

***Other considerations***

* For this case study, we only consider main wallet keys to be capable of interactively co-signing MuSig keys.

### 3.1.1. Determine different signing scenarios and their likelihoods.

* Sketch out different signing scenarios and their likelihoods.

In [1]:
# List spending paths in order of likelihood.
# TODO: Implement

# Sketch out Taproot Descriptors.
# TODO: Implement



### 3.1.2. Build your taproot output.

* Construct the Taptree according to the spending paths and their likelihoods.

In [ ]:
# Generate main wallet key pairs.
# TODO: Implement


# Generate back-up wallet key pairs.
# TODO: Implement


# 3-of-3 main key (MuSig public key).
# TODO: Implement


# Tapscripts - 2 main keys & 1 backup key
tapscript_2a = # TODO: Implement
tapscript_2b = # TODO: Implement
tapscript_2c = # TODO: Implement
tapscript_2d = # TODO: Implement
delay =        # TODO: Impplement (In blocks)
        
# Tapscripts - 1 main keys & 2 backup keys
tapscript_3a = # TODO: Implement
tapscript_3b = # TODO: Implement
long_delay =   # TODO: Implement (In blocks)

# Set list of backup tapscripts.
# Suggestion: Include tapscripts with 3d timelocks first, then those with 10d timelocks
backup_tapscripts = [tapscript_2a, tapscript_2b, tapscript_2c, tapscript_2d, tapscript_3a, tapscript_3b]

# Construct taptree with huffman constructor.
multisig_taproot = # TODO: Implement

# Print Taproot Descriptor.
print(multisig_taproot.desc)

# Construct Segwit Address.
tapscript, taptweak, control_map = multisig_taproot.construct()
output_pubkey = musig_ABC.tweak_add(taptweak) 
output_pubkey_b = output_pubkey.get_bytes()
taproot_pubkey_v1 = bytes([output_pubkey_b[0] & 1]) + output_pubkey_b[1:]
segwit_address = program_to_witness(1, taproot_pubkey_v1)
print("\nSegwit Address:", segwit_address)


### 3.1.3. Test each possible spending path of your output design.

* Construct the spending transaction and test the tx validity with the `testmempoolaccept`.
* Demonstrate the delays work intended.
* Compute and compare the weight of each spend.

***Start TestNodes***

In [ ]:
test = util.TestWrapper()
test.setup(num_nodes=1)


***Generate Wallet Balance***

In [ ]:
test.nodes[0].generate(101)
balance = test.nodes[0].getbalance()
print(balance)


***Send funds from the wallet to the taproot output (Segwit Address).***

In [ ]:
# Send funds to taproot output.
txid = test.nodes[0].sendtoaddress(segwit_address, 0.5)
print("Funding tx:", txid)

# Deserialize wallet transaction.
tx = CTransaction()
tx_hex = test.nodes[0].getrawtransaction(txid)
tx.deserialize(BytesIO(bytes.fromhex(tx_hex)))
tx.rehash()

# Determine Output Index of Segwit V1 Output.
# (Wallet places change output at a random txout index.)
outputs = iter(tx.vout)
taproot_output = next(outputs)
taproot_index = 0

while (taproot_output.scriptPubKey != tapscript):
    taproot_output = next(outputs)
    taproot_index += 1
taproot_value = taproot_output.nValue


***Test taproot spend for all script paths***

In [ ]:
# Construct signing key map (pubkey-bytes: privkey)
privkey_map = # TODO: Implement

# We will set tapscripts with different delays into separate lists.
long_delays_idx = 4
three_day_delay_txs = []
ten_day_delay_txs = []

# Iterate through all tapscripts.
for idx, tapscript_to_spend in enumerate(backup_tapscripts):
    taproot_spend_tx = CTransaction()
    taproot_spend_tx.nLockTime = 0
    taproot_spend_tx.nVersion = 2
    taproot_output_point = COutPoint(tx.sha256, taproot_index)

    # Construct transaction input.
    # TODO: Implement
    
    taproot_spend_tx.vin = [tx_input]
    dest_addr = test.nodes[0].getnewaddress(address_type="bech32")    
    spk = bytes.fromhex(test.nodes[0].getaddressinfo(dest_addr)['scriptPubKey'])
    min_fee = 5000
    dest_out = CTxOut(nValue=taproot_value - min_fee, scriptPubKey=spk)
    taproot_spend_tx.vout = [dest_out]

    htv = [0, 1, 2, 3, 0x81, 0x82, 0x83]
    sighash = TaprootSignatureHash(taproot_spend_tx, [taproot_output], htv[0], 0, scriptpath=True, tapscript=tapscript_to_spend.script)

    # Construct witness for specific to each tapscript.
    # TODO: Implement

    assert_equal(
        [{'txid': taproot_spend_tx.rehash(), 'allowed': False, 'reject-reason': '64: non-BIP68-final'}],
        test.nodes[0].testmempoolaccept([taproot_spend_str])
    )

    print("Size of Transaction #{} size is {} bytes.".format(idx, len(taproot_spend_tx.serialize()))
    
    # Sort txns into different vectors by delay.
    if idx < long_delays_idx:
        three_day_delay_txs.append(taproot_spend_tx)
    else:
        ten_day_delay_txs.append(taproot_spend_tx)
        

# Rebroadcast timelocked txs 3 day delay.
test.nodes[0].generate(delay)

for tx in three_day_delay_txs:
    assert_equal(
        [{'txid': tx.rehash(), 'allowed': True}],
        test.nodes[0].testmempoolaccept([tx.serialize().hex()])
    )

for tx in ten_day_delay_txs:
    assert_equal(
        [{'txid': tx.rehash(), 'allowed': False, 'reject-reason': '64: non-BIP68-final'}],
        test.nodes[0].testmempoolaccept([tx.serialize().hex()])
    )
    
# Rebroadcast timelocked txs after 10 day delay.
test.nodes[0].generate(long_delay-delay)
          
for tx in ten_day_delay_txs:
    assert_equal(
        [{'txid': tx.rehash(), 'allowed': True}],
        test.nodes[0].testmempoolaccept([tx.serialize().hex()])
    )
          
print("Success!")
          

***Shutdown TestNode***

In [ ]:
test.shutdown()


### 3.1.4 Remove the 3-of-3 spending path.

* Consider how you could redesign the output without the 3-of-3 spending path.
* Hint: Force spending to occur along a script spending path.
* TODO: Reconstruct the segwit address.


In [ ]:
multisig_taproot = # TODO: Implement

# Construct Segwit Address.
_, taptweak, control_map = multisig_taproot.construct()
internal_pubkey = nums_point.tweak_add(taptweak) 
internal_pubkey_b = internal_pubkey.get_bytes()
taproot_pubkey_v1 = bytes([internal_pubkey_b[0] & 1]) + internal_pubkey_b[1:]
segwit_address = program_to_witness(1, taproot_pubkey_v1)
print("\nSegwit Address:", segwit_address)
